In [1]:
import pandas as pd
import numpy as np
import re
import pytz
import os
from pathlib import Path
import sys
sys.path.append("/home/jovyan/shared/service-data")

from src.clean import clean_percentage, normalize_string, standardize_column_names, clean_fiscal_yr
from src.load import load_csv_from_raw
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

base_dir = Path.cwd()
parent_dir = base_dir.parent

In [2]:
# si_2018 = load_csv_from_raw('si_2018.csv')

# si_2018.iloc[:, 19:24].head()

In [3]:
si = merge_si()
ss = merge_ss()

Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported si_all.csv to /home/jovyan/shared/service-data/outputs/utils
Exported sid_list.csv to /home/jovyan/shared/service-data/outputs/utils
Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported ss_all.csv to /home/jovyan/shared/service-data/outputs/utils


In [4]:
si = si.loc[si['service_scope_ext_or_ent']]

si_inscope = si.set_index(['fiscal_yr','service_id'])

ss_inscope = ss.set_index(['fiscal_yr', 'service_id']).merge(si_inscope['service_scope_ext_or_ent'], how='left', left_index=True, right_index=True)
ss_inscope = ss_inscope.loc[ss_inscope['service_scope_ext_or_ent'].notna()].reset_index()


In [5]:
ss = ss_inscope

In [6]:
oip_cols = [
        'os_account_registration', 
        'os_authentication', 
        'os_application', 
        'os_decision', 
        'os_issuance', 
        'os_issue_resolution_feedback', 
    ]
    
# Melt the DataFrame
maf5 = pd.melt(
    si, 
    id_vars=['fiscal_yr', 'service_id', 'department_en','department_fr', 'org_id'], 
    var_name='online_interaction_point', 
    value_vars=oip_cols,     
    value_name='activation')

print(maf5.info())
export_to_csv(data_dict={'maf5_1': maf5}, output_dir=base_dir)

# Create boolean columns for activation states
maf5['activation_y'] = (maf5['activation'] == 'Y')
maf5['activation_n'] = (maf5['activation'] == 'N')
maf5['activation_na'] = ((maf5['activation'] == 'NA') | (maf5['activation'].isnull()))

print(maf5.info())

# Group by and sum the activation columns
maf5 = maf5.groupby(['fiscal_yr', 'department_en','department_fr', 'org_id', 'service_id'])[['activation_y', 'activation_n', 'activation_na']].sum().reset_index()

# Determine conditions for online_e2e
conditions = [
    (maf5['activation_na'] == 6),  # All interaction points are NaN
    (maf5['activation_n'] > 0)      # Some interaction points are 'N'
]
choices = [None, False]

maf5['online_e2e'] = np.select(conditions, choices, default=True).astype(bool)

# remove all Nan/Nones
maf5 = maf5.dropna(subset=['online_e2e'])

# Determine department-level counts for online e2e services and all services
maf5 = maf5.groupby(['fiscal_yr', 'department_en','department_fr', 'org_id']).agg(
    online_e2e_count=('online_e2e', 'sum'),
    service_count=('service_id', 'nunique')
).reset_index()

# Determine score and associated result
maf5['maf5_score'] = (maf5['online_e2e_count']/maf5['service_count'])*100
# # maf5['maf5_result'] = pd.cut(maf5['maf5_score'], bins=score_bins, labels=score_results, right=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53382 entries, 0 to 53381
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   fiscal_yr                 53382 non-null  object
 1   service_id                53382 non-null  object
 2   department_en             53382 non-null  object
 3   department_fr             53382 non-null  object
 4   org_id                    53382 non-null  object
 5   online_interaction_point  53382 non-null  object
 6   activation                53214 non-null  object
dtypes: object(7)
memory usage: 2.9+ MB
None
Exported maf5_1.csv to /home/jovyan/shared/service-data/notebooks
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53382 entries, 0 to 53381
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   fiscal_yr                 53382 non-null  object
 1   service_id         

In [7]:
# maf2 = ss.loc[:, ['fiscal_yr', 'service_standard_id', 'department_en','department_fr', 'org_id', 'target_met']].dropna()
    
# maf2_num = maf2[maf2['target_met']=='Y'].groupby(['fiscal_yr', 'department_en','department_fr', 'org_id'])['service_standard_id'].count().reset_index()
# maf2_denom = maf2.groupby(['fiscal_yr', 'department_en','department_fr', 'org_id'])['service_standard_id'].count().reset_index()

# maf2 = pd.merge(
#     maf2_num,
#     maf2_denom,
#     suffixes=['_met','_total'],
#     on=['fiscal_yr', 'department_en','department_fr', 'org_id'],
#     how='outer'
# )

# maf2['maf2_score'] = (maf2['service_standard_id_met']/maf2['service_standard_id_total'])*100

# maf2.loc[maf2['fiscal_yr'] == '2023-2024', ['service_standard_id_total']].sum()

In [8]:


# ifoi_en = pd.read_csv(parent_dir / "inputs" / "ifoi_en.csv")
# ifoi_fr = pd.read_csv(parent_dir / "inputs" / "ifoi_fr.csv")

# ifoi_en = ifoi_en.set_index(ifoi_en.columns[0], drop=True).add_suffix('_en')
# ifoi_fr = ifoi_fr.set_index(ifoi_fr.columns[0], drop=True).add_suffix('_fr')

# ifoi_en.set_index(ifoi_en.columns[0], drop=True)
# ifoi_fr.set_index(ifoi_fr.columns[0], drop=True)

# ifoi = pd.concat([ifoi_en, ifoi_fr], axis=1)

# # Extract column lists
# en_cols = ifoi_en.columns.tolist()
# fr_cols = ifoi_fr.columns.tolist()

# # Interleave them by index
# merged_cols = [col for pair in zip(en_cols, fr_cols) for col in pair]

# # Apply new column order
# ifoi = standardize_column_names(ifoi[merged_cols].reset_index())

# ifoi